In [19]:
!pip install catboost
!pip install ipywidgets
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
# from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import  MinMaxScaler, OneHotEncoder, LabelEncoder, LabelBinarizer

%matplotlib notebook
%matplotlib notebook

    100% |████████████████████████████████| 112kB 4.4MB/s 
    100% |████████████████████████████████| 2.2MB 8.0MB/s 


In [3]:
from google.colab import files
uploaded = files.upload()

Saving german.data to german.data


In [6]:
import io

data = pd.read_csv(io.StringIO(uploaded['german.data'].decode('utf-8')), sep=' ')
data.head()

,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1.1
0,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
1,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
2,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
3,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2
4,A14,36,A32,A46,9055,A65,A73,2,A93,A101,...,A124,35,A143,A153,1,A172,2,A192,A201,1


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 21 columns):
A11     999 non-null object
6       999 non-null int64
A34     999 non-null object
A43     999 non-null object
1169    999 non-null int64
A65     999 non-null object
A75     999 non-null object
4       999 non-null int64
A93     999 non-null object
A101    999 non-null object
4.1     999 non-null int64
A121    999 non-null object
67      999 non-null int64
A143    999 non-null object
A152    999 non-null object
2       999 non-null int64
A173    999 non-null object
1       999 non-null int64
A192    999 non-null object
A201    999 non-null object
1.1     999 non-null int64
dtypes: int64(8), object(13)
memory usage: 164.0+ KB


In [8]:
data.describe()

,6,1169,4,4.1,67,2,1,1.1
count,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000
mean,20.917918,3273.362362,2.971972,2.843844,35.514515,1.406406,1.155155,1.300300
std,12.055619,2823.365811,1.118802,1.103665,11.337487,0.577639,0.362234,0.458618
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000,1.000000
25%,12.000000,1368.500000,2.000000,2.000000,27.000000,1.000000,1.000000,1.000000
50%,18.000000,2320.000000,3.000000,3.000000,33.000000,1.000000,1.000000,1.000000
75%,24.000000,3972.500000,4.000000,4.000000,42.000000,2.000000,1.000000,2.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000,2.000000


In [9]:
data.isnull().sum()

A11     0
6       0
A34     0
A43     0
1169    0
A65     0
A75     0
4       0
A93     0
A101    0
4.1     0
A121    0
67      0
A143    0
A152    0
2       0
A173    0
1       0
A192    0
A201    0
1.1     0
dtype: int64

In [10]:
data.nunique()

A11       4
6        33
A34       5
A43      10
1169    921
A65       5
A75       5
4         4
A93       4
A101      3
4.1       4
A121      4
67       53
A143      3
A152      3
2         4
A173      4
1         2
A192      2
A201      2
1.1       2
dtype: int64

In [11]:
X = data.drop('1.1', axis =1)
y = data['1.1']

cat_vars = list(X.columns)
cat_vars.remove('1169')
X[cat_vars] = X[cat_vars].apply(lambda x: x.astype('category'))
print(cat_vars)


['A11', '6', 'A34', 'A43', 'A65', 'A75', '4', 'A93', 'A101', '4.1', 'A121', '67', 'A143', 'A152', '2', 'A173', '1', 'A192', 'A201']


In [0]:
# data[cat_vars] = data[cat_vars].apply(lambda x: x.astype('category'))
# data['1169'] = data['1169'].astype(np.int16)
# data.info()

In [0]:
scaler = MinMaxScaler()
data['1169'] = scaler.fit_transform(data['1169'].values.reshape(-1,1))
data['1169'].plot.hist()
plt.show()

In [0]:
data['1.1'].value_counts()

1    699
2    300
Name: 1.1, dtype: int64

In [0]:

# pipeline = make_pipeline(OneHotEncoder(sparse=False, handle_unknown='ignore'))
# print(X.head())
y = y.astype('category')
y.cat.categories = [0,1]
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.3, stratify = y , random_state=1234)

In [16]:
# lb = LabelBinarizer()

# data.iloc[:,[2,3]].head()
# X_train = pipeline.fit_transform(X_train)
# X_validation = pipeline.transform(X_validation)
# X_train.shape
# y_train.shape
y.head()

0    1
1    0
2    0
3    1
4    0
Name: 1.1, dtype: category
Categories (2, int64): [0, 1]

In [17]:
categorical_features_indices = np.where(X.dtypes != np.float64)[0]
categorical_features_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [0]:
model = CatBoostClassifier(iterations=500, depth=3, learning_rate=0.1, loss_function='CrossEntropy')
model.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_validation, y_validation),plot=True)

In [23]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
# print(classification_report(y_validation, ohe.inverse_transform(model.predict(X_validation))))
# print(model.predict(X_validation))
y_pred = model.predict(X_validation)
print(model.score(X_validation, y_pred))
print(classification_report(y_validation, y_pred))
print(confusion_matrix(y_validation, y_pred))
print(accuracy_score(y_validation, y_pred))

1.0
             precision    recall  f1-score   support

          0       0.79      0.92      0.85       210
          1       0.70      0.42      0.53        90

avg / total       0.76      0.77      0.75       300

[[194  16]
 [ 52  38]]
0.7733333333333333
